# **1. 개발 환경 설정**



## **1.1 필수 라이브러리 설치**

In [ ]:
#!pip install --upgrade pip #pip's dependency resolver~ 나는 오류 없애는 문구 / pip 버전이 다르므로 업데이트 진행
# 업데이트 진행 후 생기는 Running pip as the 'root'~는 python버전과 pip버전이 달라 생기는 오류지만 가상 서버니까 상관 없음
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## **1.2 Import modules**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

## **1.3 Huggingface 로그인**

In [ ]:
#login 칸에 Llama2가 승인된 hugginface 계정의 token 넣어야합니다.
notebook_login()

# **2. Dataset 생성 및 준비**

## **2.1 데이터셋 로드**

In [ ]:
#요약을 시도해볼 문서를 다운받기
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

## **2.2 데이터셋 탐색**

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [ ]:
dataset['train'][0]['document']

'앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.'

# **3. Llama 파인튜닝**

In [ ]:
# # Pre trained model
model_name = "meta-llama/Llama-2-7b-hf"

# BitsAndBytessConfig 사용하여 nf4로 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True, #모델의 매개 변수를 4비트 정밀도 형식으로 불러올지 여부 선택
    bnb_4bit_quant_type= "nf4", #4비트 표현을 위한 양자화 유형 지정 nf4 = 4-bit NormalFloat Quantization을 의미
    bnb_4bit_compute_dtype= torch.float16, #계산을 위한 데이터 유형 설정 / torch.bfloat16은 정밀도와 메모리 사용 사이의 균형을 제공하는 16비트 부동 소수점 형식
    bnb_4bit_use_double_quant= False, #4비트 값에 대해 이중 양자화 사용여부 결정
)

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
#허깅페이스에서 다운을 받고 불러오는 역할
model = AutoModelForCausalLM.from_pretrained(
    model_name, # load llama
    quantization_config=bnb_config, # 양자화한 값을 모델에 적용하여 불러오도록 함
    device_map={"": 0} #auto로 설정 시 GPU로 이동되도록 함
)

model = prepare_model_for_kbit_training(model) # 양자화된 모델을 peft를 사용하여 wrapping

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(True, True)

In [ ]:
#monitering login
wandb.login(key="wandb 키를 넣으면 됩니다.")
run = wandb.init(project='Fine tuning llama-2-7B', job_type="training", anonymous="allow")

In [ ]:
#model구조
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

q_proj (Query Projection)

쿼리 벡터를 생성하는 프로젝션입니다. 어텐션 메커니즘에서 입력 토큰을 쿼리 벡터로 변환합니다. 이 벡터는 다른 토큰들과의 유사도를 계산하는 데 사용됩니다.

k_proj (Key Projection)

키 벡터를 생성하는 프로젝션입니다. 어텐션 메커니즘에서 입력 토큰을 키 벡터로 변환하며, 쿼리 벡터와의 유사도를 계산하는 데 사용됩니다.

v_proj (Value Projection)

밸류 벡터를 생성하는 프로젝션입니다. 어텐션 메커니즘에서 입력 토큰을 밸류 벡터로 변환하며, 최종 어텐션 결과를 계산하는 데 사용됩니다.

o_proj (Output Projection)

어텐션 결과를 최종 출력으로 변환하는 프로젝션입니다. 어텐션 메커니즘의 출력이 후속 레이어로 전달되기 전에 이 프로젝션을 거칩니다.

gate_proj (Gate Projection)

종종 게이트 메커니즘이나 조합 기능을 포함하는 모델에서 사용됩니다. 이 프로젝션은 다양한 입력을 게이트하고 결합하여 다음 단계로 전달합니다.

up_proj (Up Projection)

주로 모델의 피드포워드 네트워크에서 사용되는 모듈입니다. 저차원에서 고차원으로 변환하는 역할을 하며, 모델의 표현력을 향상시킵니다.

down_proj (Down Projection)

피드포워드 네트워크의 반대 과정으로, 고차원에서 저차원으로 변환합니다. 모델의 복잡성을 줄이고 계산 효율성을 높이는 데 사용됩니다.

In [ ]:
#요약할 문서, 요약 결과를 학습시키기 위한 prompt생성용 함수
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['summary'][i]))
    return prompt_list

#데이터셋에서 train data 설정
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 글을 요약해주세요:

앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.<end_of_turn>
<start_of_turn>model
올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias="none",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",#save_steps마다 모델의 가중치가 저장될 폴더경로
        max_steps=500,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0,
        save_steps=10,
        weight_decay=0.001,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        push_to_hub=True, #허깅페이스에서 저장할지 여부
        hub_model_id='name/repo_name',  # 저장할 리포지토리 이름
        hub_strategy="every_save",  # 저장할 때마다 푸시
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,#위에서 만들었던 프롬프트 생성 함수
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#코랩 L4기준 9시간 44분 소요됨
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.201900
20,1.045600
30,0.967700
40,0.960000
50,0.926900
60,0.932700
70,0.905800
80,0.882500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

Step,Training Loss
10,1.201900
20,1.045600
30,0.967700
40,0.960000
50,0.926900
60,0.932700
70,0.905800
80,0.882500
90,0.857600
100,0.881200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=500, training_loss=0.8106617813110352, metrics={'train_runtime': 35139.6967, 'train_samples_per_second': 0.911, 'train_steps_per_second': 0.014, 'total_flos': 6.531723139397714e+17, 'train_loss': 0.8106617813110352, 'epoch': 1.440922190201729})

# **4. Llama 파인튜닝 모델 저장**

In [ ]:
#trianer.train() 실행전에 미리 mount해두는 것을 추천
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#저장경로 지정 후 모델 저장
ADAPTER_MODEL = "/content/drive/MyDrive/LLM fine tuning/lora_adapter_max_step=500"
model_name = 'meta-llama/Llama-2-7b-hf'

# ADAPTER_MODEL경로로 모델 저장
trainer.model.save_pretrained(ADAPTER_MODEL)
# 토크나이저 저장도 해야됨..! 주석으로 코드 추가
#tokenizer.save_pretrained(ADAPTER_MODEL) #tokenizer의 경우 추후 llama2에서 불러올 예정이지만 필요하다면 저장 가능

In [ ]:
# 학습시킨 ADAPTER모델과 LLaMA2 모델을 병합 및 저장
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

#model.to('cpu')  # GPU 메모리가 부족하면 사용
model = model.merge_and_unload()
model.save_pretrained("/content/drive/MyDrive/LLM fine tuning/llama-2-7b-it-sum-ko_max_step=500")

# **5. Llama 한국어 요약모델 추론(Fine-tuned 모델 로드)**

In [ ]:
finetune_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/LLM fine tuning/llama-2-7b-it-sum-ko_max_step=500", device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# 프롬프트 생성 함수
def generate_prompt(doc):
    return f"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{doc}<end_of_turn>
<start_of_turn>model"""

# 파이프라인 설정
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

# 예제 문서 로드 (여기에 실제 요약할 문서를 넣으세요)
doc = dataset['test']['document'][10]

# 프롬프트 생성
prompt = generate_prompt(doc)

# 텍스트 생성
result = pipe_finetuned(prompt)

In [ ]:
result[0]["generated_text"][len(prompt):].split('<end_of_turn>')[0].strip()

'한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행하며 울산숲사랑운동과 함께 코로나19로 지친 시민들이 활력을 되찾고 대중교통 활성화로 탄소중립 실천에 기여하길 바란다 며 앞으로도 다양한 탄소중립·환경 프로젝트를 추진하여 더 많은 시민이 참여할 수 있도록 최선을 다하겠다 고 말했다.'

' 한국동서발전 한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다. 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다. 울산시민 누구나 구글플레이 또는 앱스토어에서 스마트폰 어플 워크온 을 내려받아 회원가입 후 참여할 수 있다. 챌린지 목표인 10만보·20만보·30만보를 기간 내 달성하면 추첨을 통해 교통카드 등 소정의 기념품을 증정한다. 김태규 동서발전 탄소중립실장은 초록발자국 챌린지를 통해 코로나19로 지친 울산시민들이 활력을 되찾고 대중교통 활성화로 탄소중립 실천에 기여하길 바란다 며 앞으로도 다양한 탄소중립·환경 프로젝트를 추진하여 더 많은 시민이 참여할 수 있도록 최선을 다하겠다 고 말했다.'